In [275]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import numpy as np
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import scipy.sparse as sp
from collections import Counter
import os

Plan:
- data analysis for each dataset - done in other files
- merge them
- take out 50 datapoints for validation later
- apply algorithm to label all data

- export so i can use it on DL algo


https://www.kaggle.com/code/adityarahul/fakenewsdetection

load all labelled data and merge it

In [276]:
def merge_csv_files(folder_path):
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    dataframes = [pd.read_csv(file) for file in all_files]
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

In [277]:
labelled_folder = r"C:\github\news\news\german-news\preprocess data\data_ready_for_analysis\labelled"

In [278]:
labelled_data = merge_csv_files(labelled_folder)

In [279]:
labelled_data

,news_outlet,provenance,query_keywords,creation_date,last_modified,crawl_date,author_person,author_organization,news_keywords,title,description,body,Fake News,Extreme bias,clickbait,credible,body_len,has_label,label
0,bild,https://www.bild.de/politik/ausland/politik-au...,"linke, migration, csu, cdu",2022-06-15,2022-06-15,2024-10-20,Filipp Piatov und Ralf Schuler,NaN,NaN,Sie verspottete Deutsche als Kartoffeln: Aktiv...,Schon wieder sorgt eine Personalentscheidung d...,Schon wieder sorgt eine Personalentscheidung d...,1.0,1.0,1.0,0.0,3574.0,True,"[1, 1, 1, 0]"
1,bild,https://www.bild.de/politik/ausland/politik-au...,israel,2023-11-10,2023-11-10,2024-10-20,"P. Tiede, A. Link und H.-J. Vehlewald",NaN,NaN,Reporter als Terror-Helfer: Die perfide Propag...,"Es ist der 7. Oktober 2023. Der Tag, der alles...","Es ist der 7. Oktober 2023. Der Tag, der alles...",1.0,1.0,0.0,0.0,5226.0,True,"[1, 1, 0, 0]"
2,bild,https://www.bild.de/politik/inland/politik-inl...,"fdp, csu, spd, cdu, afd, annalena baerbock, ch...",2023-06-19,2023-06-19,2024-10-20,Filipp Piatov,NaN,"CDU, CSU, die Gruenen, SPD, Wuest Hendrik, Mer...",Streit in der CDU: Umfrage-Schlacht zwischen M...,In der CDU liegen die Nerven blank! Es tobt ei...,In der CDU liegen die Nerven blank! Spaetesten...,0.0,1.0,0.0,0.0,3071.0,True,"[0, 1, 0, 0]"
3,bild,https://www.bild.de/politik/ausland/politik-au...,"israel, spd, olaf scholz",2023-10-30,2023-10-30,2024-10-20,NaN,BILD,"Gaza-Streifen, Gaza, Israel, Hamas, Nahost-Kon...",Shani Louk ermordet Scholz: Zeigt die ganze B...,Shani Louk wurde barbarisch ermordet. Das best...,Es ist das pure Grauen. Am 7. Oktober verschle...,0.0,0.0,0.0,1.0,2655.0,True,"[0, 0, 0, 1]"
4,bild,https://www.bild.de/regional/leipzig/leipzig-n...,"abschiebung, migration, cdu",2023-11-09,2023-11-09,2024-10-20,KARL KEIM,NaN,"Tunesien, Georgien, Sachsen, asylrecht, Schust...",Sachsen: Nur jede dritte Abschiebung findet wi...,In Sachsen klappt nur jede dritte Abschiebung....,Leipzig In Sachsen wurden bisher in diesem Ja...,0.0,0.0,0.0,1.0,2447.0,True,"[0, 0, 0, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,welt,https://www.welt.de/politik/ausland/article242...,"krieg, christ, ss, sp, ukraine, ns, spo, nazi,...",2022-12-12,2022-12-12,2024-10-20,NaN,NaN,"Newsteam, Russland-Ukraine-Krieg (24.2.2022), ...",Ukraine-News ++ Ukraines Verteidigungsminister...,Laut dem ukrainischen Verteidigungsminister bi...,Laut dem ukrainischen Verteidigungsminister bi...,0.0,0.0,0.0,1.0,21564.0,True,"[0, 0, 0, 1]"
311,welt,https://www.welt.de/politik/deutschland/articl...,"sp, terror, ns, spd, schi, rechtsextremismus, ss",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Newsteam, Wehrbeauftragte, Hoegl, Eva, Reichsb...",Wehrbeauftragte fordert haerteres Vorgehen geg...,Nach der Razzia gegen ein mutmassliches Terror...,Nach der Razzia gegen ein mutmassliches Terror...,0.0,0.0,0.0,1.0,972.0,True,"[0, 0, 0, 1]"
312,welt,https://www.welt.de/politik/ausland/article242...,"krieg, sp, ukraine, ns, spd, rki, schi, ss, rn...",2022-12-10,2022-12-10,2024-10-20,NaN,NaN,"Newsteam, Leopard 2, Russland-Ukraine-Krieg (2...",Leopard 2: USA fuer Lieferung deutscher Kampfp...,Kanzler Scholz will moderne westliche Kampfpan...,Kanzler Scholz will moderne westliche Kampfpan...,0.0,0.0,0.0,1.0,2616.0,True,"[0, 0, 0, 1]"
313,welt,https://www.welt.de/politik/deutschland/articl...,"terror, ss, ns",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Schindler-Frederik, Bundestag WELT, AfD, Razzi...",Reichsbuerger-Razzia: Ermittler finden 93 Waff...,Bei der bundesweiten Razzia gegen eine Reichsb...,Bei der bundesweiten Razzia gegen eine Reichsb...,0.0,0.0,0.0,1.0,1464.0,True,"[0, 0, 0, 1]"


extract 50 labelled datapoints for later validation

In [280]:
validation_data = labelled_data.sample(n=25, random_state=42)
remaining_data = labelled_data.drop(validation_data.index)

In [281]:
labels = ['Fake News', 'Extreme bias', 'clickbait', 'credible']
validation_data[labels].sum()

Fake News        2.0
Extreme bias     7.0
clickbait        9.0
credible        19.0
dtype: float64

In [282]:
labels = ['Fake News', 'Extreme bias', 'clickbait', 'credible']
remaining_data[labels].sum()

Fake News        56.0
Extreme bias     75.0
clickbait        83.0
credible        211.0
dtype: float64

load all unlabelled data and merge it

In [283]:
unlabelled_folder = r"C:\github\news\news\german-news\preprocess data\data_ready_for_analysis\unlabelled"

In [284]:
unlabelled_data = merge_csv_files(unlabelled_folder)

preprocessing

In [285]:
X = remaining_data[['title', 'description', 'body']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
y = remaining_data[['Fake News', 'Extreme bias', 'clickbait', 'credible']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [286]:
y_train[labels].sum()

Fake News        46.0
Extreme bias     63.0
clickbait        64.0
credible        166.0
dtype: float64

In [287]:
y_test[labels].sum()

Fake News       10.0
Extreme bias    12.0
clickbait       19.0
credible        45.0
dtype: float64

bag of words

In [288]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

Random Forest

In [289]:
#train
base_model = RandomForestClassifier(random_state=42, n_estimators=100)
multi_label_model = MultiOutputClassifier(base_model)
multi_label_model.fit(X_train_tfidf, y_train)

#predict
y_pred = multi_label_model.predict(X_test_tfidf)

In [290]:
# Evaluate
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=y.columns))

Classification Report:
              precision    recall  f1-score   support

   Fake News       0.00      0.00      0.00        10
Extreme bias       0.00      0.00      0.00        12
   clickbait       1.00      0.11      0.19        19
    credible       0.80      0.96      0.87        45

   micro avg       0.79      0.52      0.63        86
   macro avg       0.45      0.27      0.26        86
weighted avg       0.64      0.52      0.50        86
 samples avg       0.77      0.70      0.72        86



c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


works better but need to work on fake news and extreme bias label

In [291]:
# import numpy as np
# def multilabel_smote(X, y, smote):
#     """
#     Multilabel SMOTE: Resample each label column independently and recombine.
#     """
#     X_resampled_list = []
#     y_resampled_list = []
    
#     for label in y.columns:
#         print(f"Applying SMOTE for label: {label}")
#         X_resampled, y_resampled = smote.fit_resample(X, y[label])
#         X_resampled_list.append(X_resampled)
#         y_resampled_list.append(y_resampled)
    
#     # Rekombinieren der Daten
#     #min_len = min(len(x) for x in X_resampled_list.shape[0])
#     X_resampled = X_resampled_list[0]  # Alle Resamplings teilen dieselbe X-Datenstruktur
#     y_resampled = np.column_stack(y_resampled_list)
#     return X_resampled, pd.DataFrame(y_resampled, columns=y.columns)


In [292]:
# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = multilabel_smote(X_train_tfidf, y_train, smote)

In [293]:
# oversampler = RandomOverSampler(random_state=42)
# X_resampled, y_resampled = X_train_tfidf, y_train.copy()

# for label in labels:
#     print(f"Resampling for label: {label}")
#     X_resampled, y_resampled[label] = oversampler.fit_resample(X_resampled, y_resampled[label])

# # Modelltraining
# base_model = RandomForestClassifier(random_state=42, n_estimators=100)
# multi_label_model = MultiOutputClassifier(base_model)
# multi_label_model.fit(X_resampled, y_resampled)

# # Vorhersagen
# y_pred = multi_label_model.predict(X_test_tfidf)

# # Evaluierung
# print("Classification Report:")
# print(classification_report(y_test, y_pred, target_names=labels))

I try random oversampling

In [294]:
target_labels_os = ['Fake News', 'Extreme bias', 'clickbait']
X_resampled, y_resampled = X_train_tfidf, y_train.copy()
oversampler = RandomOverSampler(sampling_strategy=1.0, random_state=42)
max_count = y_resampled.sum().max()
print(f"Maximum count for oversampling: {max_count}") #is 166

Maximum count for oversampling: 166.0


In [295]:
# X_resampled, y_resampled = oversampler.fit_resample(X_resampled, y_resampled)
# print("Label counts after oversampling:")
# print(y_resampled.sum())

In [296]:
target_labels_os = ['Fake News', 'Extreme bias', 'clickbait']
max_count = y_resampled.sum().max()

for label in target_labels_os:
    print(f"Oversampling for label: {label}")
    
    # Filter für die aktuelle Klasse
    y_single_label = y_resampled[label]
    
    # Übersampling der aktuellen Klasse
    X_resampled, y_single_label_resampled = oversampler.fit_resample(X_resampled, y_single_label)
    
    # Aktualisieren des resampleten Labels
    y_resampled[label] = y_single_label_resampled

if not isinstance(y_resampled, pd.DataFrame):
    y_resampled = pd.DataFrame(y_resampled, columns=y_train.columns)

print("Label counts after oversampling:")
print(y_resampled.sum())

Oversampling for label: Fake News
Oversampling for label: Extreme bias
Oversampling for label: clickbait
Label counts after oversampling:
Fake News        90.0
Extreme bias    104.0
clickbait       108.0
credible        166.0
dtype: float64


In [297]:
#train
base_model = RandomForestClassifier(random_state=42, n_estimators=100)
multi_label_model = MultiOutputClassifier(base_model)
multi_label_model.fit(X_train_tfidf, y_resampled)

#predict
y_pred = multi_label_model.predict(X_test_tfidf)

In [298]:
# Evaluate
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=y.columns))

Classification Report:
              precision    recall  f1-score   support

   Fake News       0.08      0.10      0.09        10
Extreme bias       0.21      0.25      0.23        12
   clickbait       0.39      0.37      0.38        19
    credible       0.80      0.96      0.87        45

   micro avg       0.55      0.63      0.59        86
   macro avg       0.37      0.42      0.39        86
weighted avg       0.54      0.63      0.58        86
 samples avg       0.64      0.78      0.67        86



c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


i try to get equal sizes

In [303]:
oversampler = RandomOverSampler(sampling_strategy=1.0, random_state=42)

# Initialize the resampled dataset
X_resampled, y_resampled = X_train_tfidf.copy(), y_train.copy()

# Loop through each label in y_train (assuming y_train is a DataFrame)
target_labels_os = ['Fake News', 'Extreme bias', 'clickbait', "credible"]

# Apply oversampling to each label
for label in target_labels_os:
    print(f"Oversampling for label: {label}")
    
    # Create a temporary DataFrame with the label
    y_single_label = y_resampled[label]
    
    # Oversample the current label
    X_resampled, y_single_label_resampled = oversampler.fit_resample(X_resampled, y_single_label)
    
    # Update the resampled y_resampled for the current label
    y_resampled[label] = y_single_label_resampled

# Ensure y_resampled is a DataFrame if it isn't already
if not isinstance(y_resampled, pd.DataFrame):
    y_resampled = pd.DataFrame(y_resampled, columns=y_train.columns)

# Print the label counts after oversampling
print("Label counts after oversampling:")
print(y_resampled.sum())

Oversampling for label: Fake News
Oversampling for label: Extreme bias
Oversampling for label: clickbait
Oversampling for label: credible
Label counts after oversampling:
Fake News        90.0
Extreme bias    104.0
clickbait       108.0
credible        125.0
dtype: float64


In [307]:
y_pred

array([[0., 0., 0., 0.],
       [0., 1., 1., 1.],
       [1., 1., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 1., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 1.],
       [0., 0., 0., 0.],
       [0., 1., 0., 1.],
       [1., 1., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 1.],
       [1., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 1., 1., 0.],
       [1., 0., 0., 1.],
       [1., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],


In [308]:
for i in range(len(y_pred)):
    if np.sum(y_pred[i]) == 0:  # If all labels are False (0)
        # Set the first label to True (or choose another strategy)
        y_pred[i, -1] = 1 

In [309]:
y_pred

array([[0., 0., 0., 1.],
       [0., 1., 1., 1.],
       [1., 1., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 1.],
       [1., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 1.],
       [1., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 1., 1., 0.],
       [1., 0., 0., 1.],
       [1., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],


In [310]:
#train
base_model = RandomForestClassifier(random_state=42, n_estimators=100)
multi_label_model = MultiOutputClassifier(base_model)
multi_label_model.fit(X_train_tfidf, y_resampled)

#predict
y_pred = multi_label_model.predict(X_test_tfidf)

In [311]:
# Evaluate
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=y.columns))

Classification Report:
              precision    recall  f1-score   support

   Fake News       0.08      0.10      0.09        10
Extreme bias       0.21      0.25      0.23        12
   clickbait       0.39      0.37      0.38        19
    credible       0.88      0.62      0.73        45

   micro avg       0.51      0.45      0.48        86
   macro avg       0.39      0.34      0.36        86
weighted avg       0.58      0.45      0.51        86
 samples avg       0.51      0.56      0.52        86



c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


keras

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

input_shape = X_train.shape[1]
num_labels = y_train.shape[1]

# model architecture
model = Sequential([
    Dense(512, input_shape=(input_shape,), activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(num_labels, activation='sigmoid')
])

# compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.00001), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# model summary
model.summary() 

In [ ]:
history = model.fit(X_train.toarray(), y_train.values, epochs=100, validation_split=0.4, callbacks=[early_stopping])

In [ ]:
model.save('model.h5')  

# To load the model later and resume training:
from tensorflow.keras.models import load_model
model = load_model('/kaggle/input/multiclass_seq/keras/default/1/model.h5')

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test.toarray(), y_test.values)
print(f'Test Accuracy= {round(test_accuracy,2)*100}%\nTest Loss= {round(test_loss,2)}')

other

In [ ]:
oversampler = RandomOverSampler(random_state=42)
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = X_train_tfidf, y_train.copy()

for label in labels:
    print(f"Processing label: {label}")
    
    # Resampling für einzelne Klasse
    pipeline = Pipeline([
        ('oversample', oversampler),
        ('undersample', undersampler)
    ])
    X_res, y_res = pipeline.fit_resample(X_resampled, y_resampled[label])
    
    # Update X und y
    if sp.issparse(X_resampled):
        X_resampled = sp.vstack([X_resampled, X_res])
    else:
        X_resampled = np.vstack([X_resampled, X_res])
    y_resampled[label] = np.concatenate([y_resampled[label], y_res])

# Modelltraining
base_model = RandomForestClassifier(random_state=42, n_estimators=100)
multi_label_model = MultiOutputClassifier(base_model)
multi_label_model.fit(X_resampled, y_resampled)

# Vorhersagen
y_pred = multi_label_model.predict(X_test_tfidf)

# Evaluierung
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=labels))

Processing label: Fake News


ValueError: Length of values (604) does not match length of index (232)

I try smote oversampling - currently not working

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from mlsmote import *

In [ ]:
# X_train_resampled, y_train_resampled = MLSMOTE(X_train_tfidf, y_train, 100)

In [ ]:

# #X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)
# X_train_resampled, y_train_resampled = MLSMOTE(X_train_tfidf, y_train, 100)

# # Modelltraining
# base_model = RandomForestClassifier(random_state=42, n_estimators=100)
# multi_label_model = MultiOutputClassifier(base_model)
# multi_label_model.fit(X_train_resampled, y_train_resampled)

# # Vorhersagen
# y_pred = multi_label_model.predict(X_test_tfidf)

# # Evaluierung
# print("Classification Report:")
# print(classification_report(y_test, y_pred, target_names=labels))

In [ ]:
# smote = SMOTE(random_state=42)
# model = RandomForestClassifier(class_weight='balanced', random_state=42)

In [ ]:
# pipeline.fit(X_train, y_train)

In [ ]:
# pipeline = Pipeline([
#     ('smote', smote),
#     # ('model', model)
# ])

In [ ]:
# y_pred = pipeline.predict(X_test)
# print(classification_report(y_test, y_pred))